In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
pd.set_option('display.max_columns', None)

1. Data preprocessing

In [ ]:
# Load the CSV files
previous_lives_train = pd.read_csv('/content/drive/MyDrive/Nordeus Challenge/previous_lives_training_data.csv')
registration_data_train = pd.read_csv('/content/drive/MyDrive/Nordeus Challenge/registration_data_training.csv')

previous_lives_test = pd.read_csv('/content/drive/MyDrive/Nordeus Challenge/previous_lives_test_data.csv')
registration_data_test = pd.read_csv('/content/drive/MyDrive/Nordeus Challenge/registration_data_test.csv')

In [ ]:
# Check for NaN values
# previous_lives_train.isnull().sum()
# registration_data_train.isnull().sum()

# previous_lives_test.isnull().sum()
# registration_data_test.isnull().sum()

In [ ]:
# Left join previous_lives_train on registration_data_train
data = registration_data_train.merge(
    previous_lives_train,
    on='user_id',
    how='left'
)

# Left join previous_lives_test on registration_data_test
test = registration_data_test.merge(
    previous_lives_test,
    on='user_id',
    how='left'
)

In [ ]:
data.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84941 entries, 0 to 84940
Data columns (total 40 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   user_id                                       84941 non-null  int64  
 1   registration_time_utc                         84941 non-null  object 
 2   registration_platform_specific_x              84941 non-null  object 
 3   registration_country_x                        84941 non-null  object 
 4   registration_store_x                          84941 non-null  object 
 5   registration_season_day_x                     84941 non-null  int64  
 6   registration_channel_detailed_x               84941 non-null  object 
 7   registration_device_type                      84941 non-null  object 
 8   registration_device_manufacturer              81957 non-null  object 
 9   session_count                                 84941 non-null 

In [ ]:
data.describe()
test.describe()

,user_id,registration_season_day_x,session_count,playtime,number_of_devices_used,total_match_played_count,total_match_won_count,total_match_watched_count,transaction_count_iap,transaction_count_rewarded_video,tokens_spent,tokens_stash,tokens_bought,rests_stash,rests_spent,treatments_spent,morale_spent,money_stash,avg_stars_top_11_players,avg_stars_top_3_players,avg_age_top_11_players,training_count,registration_season_day_y,days_active_lifetime,transaction_count_iap_lifetime
count,86113.000000,86113.000000,86113.000000,8.611300e+04,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,86113.000000,8.611300e+04,86113.000000,86113.000000,86111.000000,86113.000000,86113.000000,86113.000000,86113.000000
mean,402773.135136,13.685518,2.225146,1.984450e+06,1.001986,1.960505,1.240846,0.456180,0.022203,1.236631,36.487522,94.505266,0.565234,329.712877,25.137145,5.606587,4.776004,4.611344e+08,4.995280,5.669593,22.782467,3.097047,14.264014,23.868011,0.603637
std,61894.428895,8.138333,2.075008,2.593264e+06,0.119157,4.065995,2.744930,1.015859,0.255003,4.132422,66.949340,125.570994,18.570693,247.434480,54.853903,29.019701,13.807123,4.478104e+08,0.504253,0.572936,1.068109,10.662130,8.088610,50.284480,5.093712
min,300004.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2167.000000,0.000000,-91.000000,0.000000,0.000000,0.000000,-2.872708e+08,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,0.000000
25%,349829.000000,7.000000,1.000000,3.250910e+05,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.000000,0.000000,155.000000,0.000000,0.000000,0.000000,5.793843e+07,4.676788,5.298933,22.000000,0.000000,7.000000,2.000000,0.000000
50%,397061.000000,13.000000,1.000000,1.125784e+06,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9.000000,84.000000,0.000000,246.000000,0.000000,0.000000,0.000000,2.660251e+08,5.037248,5.728089,23.000000,1.000000,14.000000,6.000000,0.000000
75%,456579.000000,21.000000,3.000000,2.675355e+06,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,70.000000,119.000000,0.000000,411.000000,24.000000,0.000000,6.000000,8.758415e+08,5.351164,6.067200,23.000000,4.000000,21.000000,22.000000,0.000000
max,512713.000000,28.000000,38.000000,5.679172e+07,4.000000,71.000000,38.000000,20.000000,10.000000,60.000000,8122.000000,5884.000000,3471.000000,2036.000000,838.000000,900.000000,361.000000,2.108485e+09,7.662958,9.201067,28.000000,460.000000,28.000000,683.000000,376.000000


In [ ]:
# Compare duplicated columns
  # Compare NaN values
print(data['registration_platform_specific_x'].isnull().sum(), data['registration_platform_specific_y'].isnull().sum())
print(data['registration_country_x'].isnull().sum(), data['registration_country_y'].isnull().sum())
print(data['registration_store_x'].isnull().sum(), data['registration_store_y'].isnull().sum())
print(data['registration_season_day_x'].isnull().sum(), data['registration_season_day_y'].isnull().sum())
print(data['registration_channel_detailed_x'].isnull().sum(), data['registration_channel_detailed_y'].isnull().sum())
  # Compare difference between duplicated columns
print("Registration countries in x that are not in y ")
print(np.setdiff1d(data['registration_country_x'].unique(), data['registration_country_y'].unique()))
print("Registration stores in x that are not in y ")
print(np.setdiff1d(data['registration_store_x'].unique(), data['registration_store_y'].unique()))
print("Registration countries in y that are not in a ")
print(np.setdiff1d(data['registration_country_y'].unique(), data['registration_country_x'].unique()))

0 0
0 0
0 0
0 0
0 0
Registration countries in x that are not in y 
['Bhutan' 'Comoros' 'Djibouti']
Registration stores in x that are not in y 
['Catappult']
Registration countries in y that are not in a 
['Bahamas' 'Curacao' 'Europe' 'Madagascar' 'Seychelles'
 'Sint Maarten (Dutch part)']


In [ ]:
# Delete duplicated columns

def delete_and_rename_duplicates(df):
  cols_y_indices = [i for i, col in enumerate(df.columns) if col.endswith('_y')]
  df.drop(df.iloc[:, cols_y_indices], inplace=True, axis=1)

  # Rename remaining columns
  cols_x_indices = [i for i, col in enumerate(df.columns) if col.endswith('_x')]

  for i in cols_x_indices:
    new_name = df.columns[i][:-2]
    df.rename(columns={df.columns[i] : new_name}, inplace=True)

delete_and_rename_duplicates(data)
delete_and_rename_duplicates(test)

In [ ]:
def manipulate_nan_values(df):

  # Add "Unknown" for NaN values in registration_device_manufacturer
  df.fillna({'registration_device_manufacturer': 'Unknown'}, inplace=True)

  # Add median for NaN values in avg_age_top_11_players
  df.fillna({'avg_age_top_11_players': df['avg_age_top_11_players'].median()}, inplace=True)

  device_manufacturer = df['registration_device_manufacturer'].unique().tolist()
  sorted_device_manufacturer = sorted(device_manufacturer, key=str.lower)
  for i in sorted_device_manufacturer:
    print(i)

manipulate_nan_values(data)
manipulate_nan_values(test)

A-gold
A2
ACE France
ADVAN
AGM
AIDATA
Alldocube
alps
Amazon
ANS
ANT DAO TECHNOLOGY LIMITED
ANTEMPER
Apple
Archos
asus
ASUS
Asus
ATVIO
benco
blackshark
Blackview
BLU
BMAX
Bullitt
BullittGroupLimited
Casper
Cat
Chinoe
CHUWI
Clementoni S.p.A.
Coosea
Crosscall
CUBOT
Daewoo
DOOGEE
Droi
Droidlogic
DW
ELEVATE
emdoor
EVERCOSS
Fairphone
FIH
FROG
FUJITSU
G-Tab
General Mobile
GeneralMobile
GIONEE
Google
google
Greatwall
Haier
hena
HIKING
Hisense
HMD Global
HOMETECH
HONOR
HOPESUNIN
HOTWAV
HUAWEI
HXY
Hytera
HYUNDAI
iBRIT
IDINO
iHunt
iku
IMG
incar
INFINIX
Infinix
INFINIX MOBILITY LIMITED
Innovatech
INONE
ITEL
itel
ITEL MOBILE LIMITED
Kalley
Kinstone
KLIPAD
KMG FUEGUINA S.A.
LAVA
LeaderPhone
LENOVO
Lenovo
LG Electronics
LGE
lingjue
LNMBBS
magcomm
MAJESTIC
Meizu
Micromax
Mobicel
MODIO
motorola
Multilaser
N-one
Nothing
nubia
OBLUE
OMIX
OnePlus
OPPO
ORAIMO MOBILE LIMITED
OSCAL
OUKITEL
PEAQ
Philco
Positivo
positivo
ragentek
realme
Realme
REALME P70
reeder
Reeder
rockchip
Rockchip
RUIO
Safaricom
samsung
S

In [ ]:
# Convert all data from registration_device_manufacturer to lowercase to avoid duplicates, such as: asus, google, infinix, itel lenovo, positivo, realme, reeder, rockchip, samsung, xiaomi...
data['registration_device_manufacturer'] = data['registration_device_manufacturer'].str.lower()
device_manufacturer = data['registration_device_manufacturer'].unique().tolist()
test['registration_device_manufacturer'] = test['registration_device_manufacturer'].str.lower()
device_manufacturer_test = test['registration_device_manufacturer'].unique().tolist()

# Unite one manufacturer under a unique name to avoid duplicates, such as: bullit, droi, general mobile, infinix, itel, lg, realme, tecno...
manufacturer_mapping = {
    'bullittgrouplimited' : 'bullitt',
    'generalmobile' : 'general mobile',
    'infinix mobility limited' : 'infinix',
    'itel mobile limited' : 'itel',
    'lg electronics' : 'lg',
    'lge' : 'lg',
    'realme p70' : 'realme',
    'tecno mobile' : 'tecno',
    'tecno mobile limited' : 'tecno',
    'tecno mobility limited' : 'tecno'
}
manufacturer_mapping_test = {
    'apple apple smartphones' : 'apple',
    'generalmobile' : 'general mobile',
    'infinix mobility limited' : 'infinix',
    'itel mobile limited' : 'itel',
    'lge' : 'lg',
    'lt_mobile' : 'lt',
    'tecno mobile' : 'tecno',
    'tecno mobile limited' : 'tecno',
    'tecno mobility limited' : 'tecno',
    'sky_devices' : 'sky devices',
    'smart_products_connection_sa' : 'smart products connection sa'

}
data['registration_device_manufacturer'] = data['registration_device_manufacturer'].replace(manufacturer_mapping)
device_manufacturer = data['registration_device_manufacturer'].unique().tolist()
sorted_device_manufacturer = sorted(device_manufacturer, key=str.lower)
for i in sorted_device_manufacturer:
  print(i)

print("################################")

test['registration_device_manufacturer'] = test['registration_device_manufacturer'].replace(manufacturer_mapping_test)
device_manufacturer_test = test['registration_device_manufacturer'].unique().tolist()
sorted_device_manufacturer_test = sorted(device_manufacturer_test, key=str.lower)
for i in sorted_device_manufacturer_test:
  print(i)

a-gold
a2
ace france
advan
agm
aidata
alldocube
alps
amazon
ans
ant dao technology limited
antemper
apple
archos
asus
atvio
benco
blackshark
blackview
blu
bmax
bullitt
casper
cat
chinoe
chuwi
clementoni s.p.a.
coosea
crosscall
cubot
daewoo
doogee
droi
droidlogic
dw
elevate
emdoor
evercoss
fairphone
fih
frog
fujitsu
g-tab
general mobile
gionee
google
greatwall
haier
hena
hiking
hisense
hmd global
hometech
honor
hopesunin
hotwav
huawei
hxy
hytera
hyundai
ibrit
idino
ihunt
iku
img
incar
infinix
innovatech
inone
itel
kalley
kinstone
klipad
kmg fueguina s.a.
lava
leaderphone
lenovo
lg
lingjue
lnmbbs
magcomm
majestic
meizu
micromax
mobicel
modio
motorola
multilaser
n-one
nothing
nubia
oblue
omix
oneplus
oppo
oraimo mobile limited
oscal
oukitel
peaq
philco
positivo
ragentek
realme
reeder
rockchip
ruio
safaricom
samsung
sanmu
semp
sharp
shenzhen benton technology co., ltd.
skyth
sony
spa condor electronics
stack
stg
symphony
tcl
teclast
tecno
tem
tinno
tiok
tp-link
twz
ulefone
umidigi
unihertz

In [ ]:
# Check format for other columns
print(data['registration_platform_specific'].unique()) # OK format
print(data['registration_country'].unique()) # OK format
print(data['registration_store'].unique()) # OK format
print(data['registration_channel_detailed'].unique()) # OK format
print(data['registration_device_type'].unique()) # OK format

['iOS Phone' 'Android Phone' 'iOS Tablet' 'Android Tablet' 'WebGL TE Site'
 'WebGL FB Canvas' 'UniversalWindows PC']
['France' 'Turkey' 'Belarus' 'Indonesia' 'Egypt' 'Spain' 'Brazil'
 'Germany' 'Iraq' 'Malaysia' 'Poland' 'Romania' 'Belgium' 'Israel'
 'Croatia' 'Italy' 'United Arab Emirates' 'Greece' 'Saudi Arabia'
 'Slovakia' 'Azerbaijan' 'Japan' 'Ukraine' 'Kosovo' 'India' 'Algeria'
 'United States' 'United Kingdom' 'Sweden' 'Netherlands'
 'Equatorial Guinea' 'Switzerland' 'Czech Republic' 'Austria' 'Argentina'
 'Bulgaria' 'Serbia' 'China' 'Cambodia' 'Portugal' 'Thailand' 'Suriname'
 'Paraguay' 'El Salvador' 'Tanzania, United Republic of' 'Colombia'
 'Nigeria' 'Vietnam' 'Uzbekistan' 'Moldova, Republic of' 'Canada'
 'Denmark' 'Korea, Republic of' 'Kazakhstan' 'Morocco' 'Lebanon'
 'Venezuela' 'Cyprus' 'Oman' 'Georgia' 'Albania' 'Macedonia' 'Singapore'
 'Mexico' 'Trinidad and Tobago' 'Ecuador' 'Hungary' 'Bolivia' 'Jamaica'
 "Lao People's Democratic Republic" 'Bosnia and Herzegovina' 'Sene

In [ ]:
# Convert object data types to category to optimize memory resources
object_cols = data.select_dtypes(include='object').columns
data[object_cols] = data[object_cols].astype('category')

object_cols = test.select_dtypes(include='object').columns
test[object_cols] = test[object_cols].astype('category')

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86113 entries, 0 to 86112
Data columns (total 34 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   user_id                             86113 non-null  int64   
 1   registration_time_utc               86113 non-null  category
 2   registration_platform_specific      86113 non-null  category
 3   registration_country                86113 non-null  category
 4   registration_store                  86113 non-null  category
 5   registration_season_day             86113 non-null  int64   
 6   registration_channel_detailed       86113 non-null  category
 7   registration_device_type            86113 non-null  category
 8   registration_device_manufacturer    86113 non-null  category
 9   session_count                       86113 non-null  int64   
 10  playtime                            86113 non-null  int64   
 11  number_of_devices_used      

In [ ]:
# Perform log transformation to scale extreme values in columns with numerical data types, where mean is highly sensitive to min and max values, such as session_count

def log_transformation(df):

  df['session_count'] = np.log1p(df['session_count'])
  df['total_match_played_count'] = np.log1p(df['total_match_played_count'])
  df['total_match_won_count'] = np.log1p(df['total_match_won_count'])
  df['total_match_watched_count'] = np.log1p(df['total_match_watched_count'])
  df['transaction_count_iap'] = np.log1p(df['transaction_count_iap'])
  df['transaction_count_rewarded_video'] = np.log1p(df['transaction_count_rewarded_video'])
  df['tokens_spent'] = np.log1p(df['tokens_spent'])
  df['tokens_stash'] = np.log1p(df['tokens_stash'])
  df['tokens_bought'] = np.log1p(df['tokens_bought'])
  df['rests_stash'] = np.log1p(df['rests_stash'])
  df['rests_spent'] = np.log1p(df['rests_spent'])
  df['treatments_spent'] = np.log1p(df['treatments_spent'])
  df['morale_spent'] = np.log1p(df['morale_spent'])
  df['training_count'] = np.log1p(df['training_count'])
  df['days_active_lifetime'] = np.log1p(df['days_active_lifetime'])
  df['transaction_count_iap_lifetime'] = np.log1p(df['transaction_count_iap_lifetime'])

  df.describe()

log_transformation(data)
log_transformation(test)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


2. Model building

In [ ]:
df_train = data
df_test = test

# Separate target variable from inputs in train dataset
X_train = df_train.drop(['user_id', 'days_active_first_28_days_after_registration'], axis = 1)
Y_train = df_train['days_active_first_28_days_after_registration']

# Separate target variable from inputs in test dataset
X_test = df_test.drop(['user_id'], axis=1)
user_ids = df_test['user_id']

cat_features = [
    'registration_time_utc',
    'registration_platform_specific',
    'registration_country',
    'registration_store',
    'registration_channel_detailed',
    'registration_device_type',
    'registration_device_manufacturer',
    'registration_date',
    'is_payer_lifetime',
    'is_rewarded_video_watcher_lifetime'
]

# Initialize and train model
model = CatBoostRegressor(iterations=1000,
                          learning_rate=0.05,
                          depth=10,
                          cat_features=[X_train.columns.get_loc(c) for c in cat_features],
                          verbose=100)

model.fit(X_train, Y_train)

# Predict model
y_pred = model.predict(X_test)

# Create resulting dataset
results = pd.DataFrame({
    'user_id': user_ids,
    'predicted_days_active_first_28_days_after_registration': y_pred
})

# Save to CSV
results.to_csv('days_active_first_28_days_after_registration_predictions.csv', index=False)


0:	learn: 9.4481621	total: 1.03s	remaining: 17m 13s
100:	learn: 6.9287152	total: 45.3s	remaining: 6m 43s
200:	learn: 6.6891463	total: 1m 31s	remaining: 6m 5s
300:	learn: 6.4624812	total: 2m 17s	remaining: 5m 18s
400:	learn: 6.2662806	total: 3m 2s	remaining: 4m 32s
500:	learn: 6.0945316	total: 3m 48s	remaining: 3m 47s
600:	learn: 5.9521205	total: 4m 35s	remaining: 3m 2s
700:	learn: 5.8275758	total: 5m 21s	remaining: 2m 16s
800:	learn: 5.7020879	total: 6m 7s	remaining: 1m 31s
900:	learn: 5.5949884	total: 6m 54s	remaining: 45.6s
999:	learn: 5.4855996	total: 7m 40s	remaining: 0us
